Извлечем признаки:
- на основе даты
- на основе предыдущих значений ряда (агрегаты)

In [17]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from datetime import timedelta
one_day = timedelta(days=1)
cal = calendar()
#инициализировали объект календарь

In [18]:
train = pd.read_csv("train.csv", index_col='Month', parse_dates=[0])
test = pd.read_csv("test.csv", index_col='Month', parse_dates=[0])
train['date'] = train.index.values
test['date'] = test.index.values
#train['date'] = train[['dt', 'Time']].apply(lambda x: x[0]+" "+x[1], 1).astype('datetime64')
#test['date'] = test[['dt', 'Time']].apply(lambda x: x[0]+" "+x[1], 1).astype('datetime64')
train.head(3)

,Passengers,date
Month,,
1949-01-01,112,1949-01-01
1949-02-01,118,1949-02-01
1949-03-01,132,1949-03-01


In [19]:
def extract_date_feauters(data):
    data['year'] = data['date'].dt.year#apply(lambda x: x.year, 1)
    data['month'] = data['date'].dt.month
    data['dayofmonth'] = data['date'].dt.day
    data['quarter'] = data['date'].dt.quarter
    data['dayofyear'] = data['date'].dt.dayofyear
    data['weekofyear'] = data['date'].dt.weekofyear
    data['dayofweek'] = data['date'].dt.dayofweek
    data['hour'] = data['date'].dt.hour
    return data

In [20]:
train = extract_date_feauters(train)
test = extract_date_feauters(test)

In [21]:
test.head(3)

,Passengers,date,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,hour
Month,,,,,,,,,,
1959-11-01,362,1959-11-01,1959,11,1,4,305,44,6,0
1959-12-01,405,1959-12-01,1959,12,1,4,335,49,1,0
1960-01-01,417,1960-01-01,1960,1,1,1,1,53,4,0


Добавим признаки на основе прошлых значений ряда

In [22]:
def exponential_smoothing(series, alpha):
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

In [23]:
def extra_features(df):
    df['t1'] = df['Passengers'].shift(1)
    df['t2'] = df['Passengers'].shift(2)
    df['t3'] = df['Passengers'].shift(3)
    # добавили сглаживание (3 вида - просто для коллекции)
    df['exp_0_01'] = exponential_smoothing(df['Passengers'], 0.01)
    df['exp_0_03'] = exponential_smoothing(df['Passengers'], 0.03)
    df['exp_0_08'] = exponential_smoothing(df['Passengers'], 0.08)
    # добавим разности - 1-й порядок
    #df['diff1'] = 0
    #df['diff1'].iloc[1:] = df.iloc[:-1,0]-df.iloc[1:,0]
    return df.fillna(0)

In [24]:
train = extra_features(train)
test = extra_features(test)

In [25]:
train.drop(['date'], 1).to_csv("train_prepared.csv")
test.drop(['date'], 1).to_csv("test_prepared.csv")

Сохранили датасеты и теперь готовы к моделированию